# WorkFlow
## Load the data
## Clean the data
## test modelling
## modelling

## Load the data

In [53]:
import pandas as pd

In [54]:
data_0 = pd.read_csv('./data/raw/data-1.csv')
data_1 = pd.read_csv('./data/raw/data-2.csv')

In [55]:
data_0 = data_0[data_0['location'] == 'Sri Lanka']
data_1 = data_1[data_1['Country/Region'] == 'Sri Lanka']

## Clean the data

In [56]:
new_recovered_list = []
for row_index,(info) in enumerate(data_1['Recovered']):
    if row_index == 0:
        new_recovered_list.append(info)
    else:
        new_recovered_list.append(info - data_1['Recovered'].iloc[row_index - 1])

In [57]:
new_recovered = new_recovered_list
total_recovered = data_1['Recovered']
new_cases = data_0['new_cases']
total_cases = data_0['total_cases']
total_deaths = data_0['total_deaths']
new_deaths = data_0['new_deaths']
new_tests = data_0['new_tests']
total_tests = data_0['total_tests']
positive_rate = data_0['positive_rate']
total_vaccinations = data_0['total_vaccinations']
new_vaccinations = data_0['new_vaccinations']

In [58]:
new_tests = new_tests.fillna(new_tests.median())
total_tests = total_tests.fillna(total_tests.median())
positive_rate = positive_rate.fillna(positive_rate.median())
total_vaccinations = total_vaccinations.fillna(total_vaccinations.median())
new_vaccinations = new_vaccinations.fillna(new_vaccinations.median())

In [59]:
new_cases = new_cases.fillna(new_cases.mean())
total_cases = total_cases.fillna(total_cases.mean())
total_deaths = total_deaths.fillna(total_deaths.mean())
new_deaths = new_deaths.fillna(new_deaths.mean())
new_tests = new_tests.fillna(new_tests.mean())
total_cases = total_cases.fillna(total_cases.mean())
total_tests = total_tests.fillna(total_tests.mean())
positive_rate = positive_rate.fillna(positive_rate.mean())
total_vaccinations = total_vaccinations.fillna(total_vaccinations.mean())
new_vaccinations = new_vaccinations.fillna(new_vaccinations.mean())
total_recovered = total_recovered.fillna(total_recovered.mean())

In [60]:
dates_3 = []
for date in data_1['ObservationDate']:
    date = str(date).split('/')
    date = int(date[2] + date[0] + date[1])
    dates_3.append(int(date))
data_1['ObservationDate'] = dates_3

In [61]:
dates_1 = []
for date in data_0['date']:
    date = str(date).replace('-','')
    dates_1.append(int(date))
data_0['date'] = dates_1

In [62]:
len(new_recovered),len(total_recovered)

(462, 462)

In [63]:
infos = [new_cases,total_cases,total_deaths,new_deaths,new_tests,total_cases,total_tests,positive_rate,total_vaccinations,new_vaccinations,total_recovered,new_recovered]
infos_names = ['new_cases','total_cases','total_deaths','new_deaths','new_tests','total_cases','total_tests','positive_rate','total_vaccinations','new_vaccinations','total_recovered','new_recovered']

In [64]:
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor

In [65]:
import numpy as np

In [66]:
results = {}

In [67]:
data_1['ObservationDate']

268       20200127
321       20200128
374       20200129
430       20200130
497       20200131
            ...   
281633    20210428
282397    20210429
283161    20210430
283925    20210501
284689    20210502
Name: ObservationDate, Length: 462, dtype: int64

In [68]:
for info,name in zip(infos,infos_names):
    try:
        model = RandomForestRegressor().fit(np.array(data_0['date']).reshape(-1,1),info)
        date = np.array([20210509])
        results[name] = list(np.round(model.predict(date.reshape(-1,1))))[0]
    except:
        model = RandomForestRegressor().fit(np.array(data_1['ObservationDate']).reshape(-1,1),info)
        date = np.array([20210509])
        results[name] = list(np.round(model.predict(date.reshape(-1,1))))[0]

In [69]:
{'new_cases': 1911.0,
 'total_cases': 120184.0,
 'total_deaths': 754.0,
 'new_deaths': 16.0,
 'new_tests': 5053.0,
 'total_tests': 622148.0,
 'positive_rate': 0.0,
 'total_vaccinations': 1089464.0,
 'new_vaccinations': 17831.0,
 'total_recovered': 97235.0,
 'new_recovered': 1073.0}

{'new_cases': 1911.0,
 'total_cases': 120184.0,
 'total_deaths': 754.0,
 'new_deaths': 16.0,
 'new_tests': 5053.0,
 'total_tests': 622148.0,
 'positive_rate': 0.0,
 'total_vaccinations': 1089464.0,
 'new_vaccinations': 17831.0,
 'total_recovered': 97235.0,
 'new_recovered': 1073.0}

In [70]:
results

{'new_cases': 1910.0,
 'total_cases': 120165.0,
 'total_deaths': 754.0,
 'new_deaths': 17.0,
 'new_tests': 5239.0,
 'total_tests': 570317.0,
 'positive_rate': 0.0,
 'total_vaccinations': 1093931.0,
 'new_vaccinations': 17686.0,
 'total_recovered': 97323.0,
 'new_recovered': 1001.0}

In [71]:
file_names = []

In [72]:
for info,name in zip(infos,infos_names):
    if name == 'total_recovered' or name == 'new_recovered':
        df = pd.DataFrame({'date':data_1['ObservationDate'],name:info})
        df.to_csv(f'./data/cleaned/{name}.csv',index=False)
        file_names.append([f'./data/cleaned/{name}.csv',name])
    else:
        df = pd.DataFrame({'date':data_0['date'],name:info})
        df.to_csv(f'./data/cleaned/{name}.csv',index=False)
        file_names.append([f'./data/cleaned/{name}.csv',name])

In [73]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor,AdaBoostRegressor,BaggingRegressor,RandomForestRegressor
from sklearn.svm import SVR

In [74]:
models = [
    ['DecisionTreeRegressor',DecisionTreeRegressor],
    ['GradientBoostingRegressor',GradientBoostingRegressor],
    ['AdaBoostRegressor',AdaBoostRegressor],
    ['BaggingRegressor',BaggingRegressor],
    ['RandomForestRegressor',RandomForestRegressor],
    ['SVR',SVR],
]

In [75]:
results = []
for model in models:
    results = []
    model_name = model[0]
    for file in file_names:
        data = pd.read_csv(file[0])
        X = data['date']
        X = np.array(X).reshape(-1,1)
        y = data[file[1]]
        model_ = model[1]()
#         model_ = DecisionTreeRegressor(max_features='sqrt', splitter='random')
        model_.fit(X,y)
        results.append(model_.score(X,y))               
    print(model_name)
    print(np.mean(results))
# 0.9619995758307031

DecisionTreeRegressor
0.9619995758307031
GradientBoostingRegressor
0.9556697867917268
AdaBoostRegressor
0.8994385596270581
BaggingRegressor
0.9518078064944476
RandomForestRegressor
0.9550801112133459
SVR
-0.17987967405888142


In [76]:
# DecisionTreeRegressor Wins !!

In [77]:
grid_search = {
    'criterion':['mse','friedman_mse','mae','poissin'],
    'splitter':['best','random'],
    'max_features':['auto','sqrt','log2'],
}

In [78]:
model = DecisionTreeRegressor()

In [79]:
from sklearn.model_selection import *
from sklearn.metrics import *

In [80]:
model = GridSearchCV(model,grid_search,n_jobs=-1,cv=5,verbose=5)

In [81]:
model.fit(X,y)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


/home/indika/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:925: UserWarning: One or more of the test scores are non-finite: [-2.58231852 -1.61125877 -2.58231852 -1.86111536 -2.58231852 -3.47496656
 -2.58231852 -1.19710449 -2.58231852 -1.4454228  -2.58231852 -3.50269364
 -2.58231852 -2.55459145 -2.58231852 -3.50763096 -2.58231852 -2.30728359
         nan         nan         nan         nan         nan         nan]
  category=UserWarning


GridSearchCV(cv=5, estimator=DecisionTreeRegressor(), n_jobs=-1,
             param_grid={'criterion': ['mse', 'friedman_mse', 'mae', 'poissin'],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'splitter': ['best', 'random']},
             verbose=5)

In [82]:
model.best_estimator_

DecisionTreeRegressor(criterion='friedman_mse', max_features='auto',
                      splitter='random')

In [83]:
model.score(X,y)

0.9615551409924294

In [84]:
model = DecisionTreeRegressor(max_features='sqrt', splitter='random')

In [85]:
import pickle

In [88]:
file_names

[['./data/cleaned/new_cases.csv', 'new_cases'],
 ['./data/cleaned/total_cases.csv', 'total_cases'],
 ['./data/cleaned/total_deaths.csv', 'total_deaths'],
 ['./data/cleaned/new_deaths.csv', 'new_deaths'],
 ['./data/cleaned/new_tests.csv', 'new_tests'],
 ['./data/cleaned/total_cases.csv', 'total_cases'],
 ['./data/cleaned/total_tests.csv', 'total_tests'],
 ['./data/cleaned/positive_rate.csv', 'positive_rate'],
 ['./data/cleaned/total_vaccinations.csv', 'total_vaccinations'],
 ['./data/cleaned/new_vaccinations.csv', 'new_vaccinations'],
 ['./data/cleaned/total_recovered.csv', 'total_recovered'],
 ['./data/cleaned/new_recovered.csv', 'new_recovered']]

In [86]:
for file in file_names:
    data = pd.read_csv(file[0])
    X = data['date']
    X = np.array(X).reshape(-1,1)
    y = data[file[1]]
    model.fit(X,y)
    print(file[1])
    print(model.predict(np.array([20210509]).reshape(-1,1)))
    pickle.dump(model,open(f'./API/models/{file[1]}.pkl','wb'))

new_cases
[1914.]
total_cases
[121338.]
total_deaths
[764.]
new_deaths
[19.]
new_tests
[2827.]
total_cases
[121338.]
total_tests
[287864.]
positive_rate
[0.03]
total_vaccinations
[1105342.]
new_vaccinations
[17249.]
total_recovered
[97848.]
new_recovered
[1370.]


In [87]:
file_names

[['./data/cleaned/new_cases.csv', 'new_cases'],
 ['./data/cleaned/total_cases.csv', 'total_cases'],
 ['./data/cleaned/total_deaths.csv', 'total_deaths'],
 ['./data/cleaned/new_deaths.csv', 'new_deaths'],
 ['./data/cleaned/new_tests.csv', 'new_tests'],
 ['./data/cleaned/total_cases.csv', 'total_cases'],
 ['./data/cleaned/total_tests.csv', 'total_tests'],
 ['./data/cleaned/positive_rate.csv', 'positive_rate'],
 ['./data/cleaned/total_vaccinations.csv', 'total_vaccinations'],
 ['./data/cleaned/new_vaccinations.csv', 'new_vaccinations'],
 ['./data/cleaned/total_recovered.csv', 'total_recovered'],
 ['./data/cleaned/new_recovered.csv', 'new_recovered']]